In [15]:
from rouge_score import rouge_scorer

In [4]:
import pandas as pd 
import numpy as np
import nltk
from nltk.corpus import stopwords
from tqdm._tqdm_notebook import tqdm_notebook
import warnings
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
warnings.filterwarnings('ignore')
tqdm_notebook.pandas()
nltk.download('stopwords')


C:\Users\ryzhk\AppData\Local\Temp\ipykernel_740\4037459634.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ryzhk\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ryzhk\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [8]:
df = pd.read_csv('bi-lstm-result.csv')

In [16]:
rouge_metrics = rouge_scorer.RougeScorer(['rouge1', 'rougeL','rouge2'], use_stemmer=True)

In [12]:
df.head()

,Article,Original Summary,Model Output
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,the airline s air force force force force for...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,the man was caught by a man in a south africa...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,the club s owner has been a new contract for ...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,the italian striker has been a move to the cl...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...",the former star was a new york for the first ...


In [14]:
df['Model Output'][5]

' the dog was taken to the hospital in the south africa the south africa was a local of the wildlife park in the south africa the south africa was a local of the wildlife park in the area of the park'

In [56]:
print(df['highlights'][0])

Bishop John Folda, of North Dakota, is taking time off after being diagnosed .
He contracted the infection through contaminated food in Italy .
Church members in Fargo, Grand Forks and Jamestown could have been exposed .


In [57]:
df.shape

(287113, 3)

In [67]:
df_need = df[:1000]

In [ ]:
PERCENT_RATIO = 0.2
STOPWORDS = stopwords.words('english')

#### text rank

In [ ]:
from summa.summarizer import summarize as textRankSumm

In [78]:
df_need['text_rank'] = df_need['article'].progress_apply(lambda x:summarize(x,ratio = PERCENT_RATIO,additional_stopwords = STOPWORDS))

  0%|          | 0/1000 [00:00<?, ?it/s]

#### LexRank

In [124]:
from lexrank import LexRank

lxr = LexRank(df_need['article'],stopwords=STOPWORDS)
df_need['article_sentences'] = df_need['article'].apply(lambda x:sent_tokenize(x))


In [127]:
df_need['lex_rank'] = df_need['article_sentences']\
.progress_apply(lambda x: lxr\
                .get_summary(x,
                             summary_size=round(PERCENT_RATIO*len(x))
                            )
               )

  0%|          | 0/1000 [00:00<?, ?it/s]

In [128]:
def connect(x:list[str])->str:
    return ''.join(x)

df_need['lex_rank'] = df_need['lex_rank'].apply(conect)

#### Evaluate



In [134]:
print(df_need['text_rank'][0],'\n\n',df_need['highlights'][0])

The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October.
Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . 

 Bishop John Folda, of North Dakota, is taking time off after being diagnosed .
He contracted the infection through contaminated food in Italy .
Church members in Fargo, Grand Forks and Jamestown could have been exposed .


In [141]:
abba = rouge_metrics.score(df_need['highlights'][0],df_need['text_rank'][0])

In [144]:
abba['rougeL'].fmeasure

0.28865979381443296

In [153]:
df_need.columns

Index(['id', 'article', 'highlights', 'text_rank', 'article_sentences',
       'lex_rank'],
      dtype='object')

In [164]:

df_need['text_rank_scores'] = df_need[['highlights','text_rank']]\
.apply(lambda x: rouge_metrics.score(x['highlights'],x['text_rank'])['rougeL'].fmeasure,axis = 1)

df_need['lex_rank_scores'] = df_need[['highlights','lex_rank']]\
.apply(lambda x: rouge_metrics.score(x['highlights'],x['lex_rank'])['rougeL'].fmeasure,axis = 1)

In [162]:
df_need['text_rank_scores']

0      0.288660
1      0.126582
2      0.224924
3      0.119403
4      0.099099
         ...   
995    0.140000
996    0.176166
997    0.194175
998    0.283333
999    0.183673
Name: text_rank_scores, Length: 1000, dtype: float64

In [165]:
df_need['lex_rank_scores'] 

0      0.241379
1      0.189189
2      0.139394
3      0.216749
4      0.118644
         ...   
995    0.126984
996    0.251429
997    0.202899
998    0.216216
999    0.234234
Name: lex_rank_scores, Length: 1000, dtype: float64

In [31]:
pip install scipy

     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.6 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.6 kB 162.5 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.6 kB 281.8 kB/s eta 0:00:01
     -------------------------------------- 60.6/60.6 kB 357.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/45.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/45.9 MB 1.3 MB/s eta 0:00:36
   ---------------------------------------- 0.2/45.9 MB 2.1 MB/s eta 0:00:22
   ---------------------------------------- 0.3/45.9 MB 2.1 MB/s eta 0:00:22
   ---------------------------------------- 0.4/45.9 MB 1.9 MB/s eta 0:00:25
   ---------------------------------------- 0.5/45.9 MB 2.0 MB/s eta 0:00:23
    --------------------------------------- 0.6/45.9 MB 2.1 MB/s eta 0:00:22
    --------------------------------------- 0.7/45.9 MB 2.2 MB/s eta 0:00:21
    --

In [32]:
import numpy as np
from scipy import stats

def calculate_confidence_interval(data, confidence=0.95):
    mean = np.mean(data)
    n = len(data)
    stderr = stats.sem(data)
    interval = stderr * stats.t.ppf((1 + confidence) / 2., n - 1)
    return mean - interval, mean ,mean + interval

In [171]:
calculate_confidence_interval(df_need['lex_rank_scores'])

(0.15897700141626406, 0.16372254932094313, 0.1684680972256222)

In [172]:
calculate_confidence_interval(df_need['text_rank_scores'])

(0.1724774936049619, 0.17748832458745759, 0.18249915556995328)

## Bi-LSTM

In [17]:
import pandas as pd
df_bi = pd.read_csv('C:/Users/ryzhk/Documents/Projects/TestSummarizationDiploma/bi-lstm-result.csv')

In [27]:
df_bi['scores'] = df_bi[['Original Summary','Model Output']]\
.apply(lambda x: rouge_metrics.score(x['Original Summary'],x['Model Output'])['rougeL'].fmeasure,axis = 1)


In [33]:
calculate_confidence_interval(df_bi['scores'])

(0.10552818004568913, 0.11036783983213726, 0.11520749961858538)

In [34]:
df_bi['scores1'] = df_bi[['Original Summary','Model Output']]\
.apply(lambda x: rouge_metrics.score(x['Original Summary'],x['Model Output'])['rouge1'].fmeasure,axis = 1)

df_bi['scores2'] = df_bi[['Original Summary','Model Output']]\
.apply(lambda x: rouge_metrics.score(x['Original Summary'],x['Model Output'])['rouge2'].fmeasure,axis = 1)


In [35]:
calculate_confidence_interval(df_bi['scores2'])

(0.018284537408088657, 0.020566170806652143, 0.02284780420521563)

In [36]:
calculate_confidence_interval(df_bi['scores1'])

(0.13100073604655407, 0.13750704710536849, 0.1440133581641829)